In [13]:
# Install necessary libraries
!pip install pandas scikit-learn transformers torch datasets

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score , precision_score, recall_score

# Step 1: Load and Preprocess Dataset
def load_data(file_path):
    """Load dataset from a CSV file and check for required columns."""
    data = pd.read_excel(file_path)
#     data = data.sample(frac=0.04, random_state=42)  # Random 40%

    if 'tweets' not in data.columns:
        raise KeyError("The dataset must contain a 'text' column with tweet data.")
    if 'labels' not in data.columns:
        print("Warning: No 'label' column found. Assuming this is the unlabeled data.")
        data['labels'] = None  # Add a label column with NaN values if missing
    return data


def preprocess_text(df):
    """Preprocess text data."""
    df['tweets'] = df['tweets'].str.lower()
    df['tweets'] = df['tweets'].str.replace(r"http\S+|www\S+|https\S+", '', regex=True)  # remove URLs
    df['tweets'] = df['tweets'].str.replace(r'\@\w+|\#', '', regex=True)  # remove mentions and hashtags
    df['tweets'] = df['tweets'].str.replace(r'[^A-Za-z\s]', '', regex=True)  # remove non-alphanumeric chars
    return df

# Step 2: Define Labeled and Unlabeled Dataset
from sklearn.model_selection import train_test_split

def split_data(data, label_column='labels', labeled_size=0.2, unlabeled_size=0.6, test_size=0.2):
    """Split dataset into labeled, unlabeled, and test data."""
    # Ensure the sum of labeled_size, unlabeled_size, and test_size equals 1
    assert labeled_size + unlabeled_size + test_size == 1, "The sizes must sum to 1."
    
    # Treat all data as unlabeled, including the rows with labels
    
    
    # Split the data into train (80%) and test (20%) sets
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)
    
    # Now, from the train_data (80%), we select 20% as labeled data
    labeled_data, unlabeled_data = train_test_split(train_data, test_size=0.75, random_state=42)
    
    return labeled_data,  test_data , unlabeled_data




# Step 3: Initialize Model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def tokenize(batch):
    # Convert each item in the 'tweets' list to string using list comprehension
    batch['tweets'] = [str(item) for item in batch['tweets']]  
    return tokenizer(batch['tweets'], padding='max_length', truncation=True, max_length=128)
# Step 4: Training Arguments
def setup_training_args():
    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=3,
        weight_decay=0.01,
        evaluation_strategy="epoch"
    )
    return training_args

# Step 5: Initialize Trainer and Model Training
# ... (your existing code) ...

# Step 5: Initialize Trainer and Model Trainin
from datasets import Dataset  
def train_model(train_data, eval_data, model, training_args):
    # Convert DataFrames to Hugging Face Datasets
    train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
    eval_dataset = Dataset.from_pandas(eval_data.reset_index(drop=True))
    
    # Tokenize datasets
    train_dataset = train_dataset.map(tokenize, batched=True)
    eval_dataset = eval_dataset.map(tokenize, batched=True)
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,  # Use Hugging Face Datasets
        eval_dataset=eval_dataset,    # Use Hugging Face Datasets
        compute_metrics=compute_metrics
    )
    trainer.train()
    return trainer

# ... (rest of your code) ...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Step 6: Iterative Data Harvesting - Semi-Supervised Learning
# Step 6: Iterative Data Harvesting - Semi-Supervised Learning
def predict_with_confidence(trainer, data, threshold=0.9, batch_size=16):  # Added batch_size parameter
    """Predicts labels for unlabeled data with high confidence."""
    # Check if 'data' has any rows
    if data.empty:
        return pd.DataFrame(columns=data.columns)  # Return empty DataFrame

    # Convert each item in the 'tweets' list to string using list comprehension
    # and filter out any empty strings
    valid_tweets = [str(item) for item in data['tweets'].tolist() if str(item).strip()]
    
    # Check if there are any valid tweets
    if not valid_tweets:
        return pd.DataFrame(columns=data.columns)  # Return empty DataFrame
    
    # Process data in batches to reduce memory usage
    all_high_confidence_indices = []
    for i in range(0, len(valid_tweets), batch_size):
        batch_tweets = valid_tweets[i : i + batch_size]
        inputs = tokenizer(batch_tweets, return_tensors="pt", truncation=True, padding=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=1)
        high_confidence_indices = (probs.max(dim=1).values > threshold).nonzero(as_tuple=True)[0]
        all_high_confidence_indices.extend(high_confidence_indices.cpu().numpy() + i)  # Adjust indices

    # If no high confidence predictions, return empty DataFrame
    if not all_high_confidence_indices:
        return pd.DataFrame(columns=data.columns)

    return data.iloc[all_high_confidence_indices]

def iterative_harvesting(trainer, labeled_data, unlabeled_data, iterations=1, threshold=0.9):
    for iteration in range(iterations):
        high_conf_data = predict_with_confidence(trainer, unlabeled_data, threshold)
        print(len(labeled_data))
        labeled_data = pd.concat([labeled_data, high_conf_data])
        print(len(labeled_data))
        unlabeled_data = unlabeled_data.drop(high_conf_data.index)
        trainer.train()  # Retrain on expanded labeled data
    return labeled_data, trainer

# Step 7: Zero-Shot Learning Integration (optional)
from transformers import pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["depression", "no depression"]



# Step 8: Evaluation Metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)  # Get the predicted class labels
    acc = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='binary')
    recall = recall_score(labels, preds, average='binary')
    f1 = f1_score(labels, preds, average='binary')
    
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1}



# Main Pipeline Function
def evaluate_model(trainer, eval_data):
    # Re-tokenize eval_data to ensure consistency
#     print("a")
    eval_dataset = Dataset.from_pandas(eval_data.reset_index(drop=True))
#     print("b")
    eval_dataset = eval_dataset.map(tokenize, batched=True)
#     print("c")
    # Run evaluation
    final_results = trainer.evaluate(eval_dataset=eval_dataset)
#     print("s")
    return final_results

def run_pipeline(file_path):
    # Load and preprocess data
    data = load_data(file_path)
    data = preprocess_text(data)

    # Define labeled and unlabeled datasets
    train_data, eval_data, unlabeled_data = split_data(data)
    
    print(train_data)
    print(eval_data)
    print(unlabeled_data)
    # Initialize training arguments and trainer
    training_args = setup_training_args()
    print("training started.")
    trainer = train_model(train_data, eval_data, model, training_args)
    print("training ended.")
    # Iterative Data Harvesting
    print("Iterative Data Harvesting started.")
    final_labeled_data, final_trainer = iterative_harvesting(trainer, train_data, unlabeled_data)
    print("Iterative Data Harvesting ended.")

    # Final Evaluation
    print("Final Evaluation started.")
    
    final_results = evaluate_model(final_trainer, eval_data)
    print("Final model results:")
    print(f"Accuracy: {final_results['eval_accuracy']:.4f}")
    print(f"Precision: {final_results['eval_precision']:.4f}")
    print(f"Recall: {final_results['eval_recall']:.4f}")
    print(f"F1 Score: {final_results['eval_f1']:.4f}")
    return final_labeled_data, final_results

# Run the pipeline with your dataset file path
file_path = 'newenglish.xlsx'
run_pipeline(file_path)

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Please use a different name to suppress this warning.
A parameter name that contains `beta` will be renamed internally to `bias`. Please use a different name to suppress this warning.
A parameter name that contains `gamma` will be renamed internally to `weight`. Pl

                                                  tweets  labels
1718   baccording climate change negatively impacts p...       1
22691  bitxexxs okay help took years comfortable enou...       0
51306  bsomeone labeled pleaser guess xfxfxaxbxfxfxfx...       0
40337   really glad that vote hunter biden last election       0
38807  laugh innocent infectiousn will stop until jus...       0
...                                                  ...     ...
37076       glad lori michael stopped weekly photoshoots       0
31466  bthis easter sunday been amazing risen feel ho...       0
47469  bxfxfxx mood increased better past minutes ana...       0
7475   almost every measure collective mental health ...       1
39251  bwishing happy hopeful easter those celebratin...       0

[481 rows x 2 columns]
                                                  tweets  labels
53531      bglad some isley brothers appreciation lately       0
19227  between this other research saying that depres...       1
4

/home/abcd/anaconda3/envs/abcd/lib/python3.8/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/481 [00:00<?, ? examples/s]

...

training ended.
Iterative Data Harvesting started.
481
1791


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.721929,0.836100,0.715686,0.874251,0.787062
2,No log,0.827647,0.836100,0.709524,0.892216,0.790451
3,No log,0.723206,0.865145,0.807229,0.802395,0.804805


1791
1891


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.851684,0.869295,0.846667,0.760479,0.801262
2,No log,0.917639,0.863071,0.848276,0.736527,0.788462


KeyboardInterrupt: 